In [ ]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

In [ ]:
def read_data():
    ecfp = pickle.load( open( "ecfp.p", "rb" ) )
    new_df  = pd.read_csv("zinc.csv")
    new_df['ecfp'] = ecfp
    gene_list_up = pd.read_csv('gene_lists/PC3_multi_task_ecfp_gene_list_up.txt',header=None ,delimiter = "\t")
    genes_up = gene_list_up[0].tolist()
    
    gene_list_dn = pd.read_csv('gene_lists/PC3_multi_task_ecfp_gene_list_dn.txt',header=None ,delimiter = "\t")
    genes_dn = gene_list_dn[0].tolist()
    return new_df, ecfp, genes_up, genes_dn

In [ ]:
new_df, ecfp, genes_up, genes_dn = read_data()

In [ ]:
def read_model():
    model_up = keras.models.load_model("multi-task_ecfp_models/PC3_multi_task_model_ecfp_up.h5")
    model_dn = keras.models.load_model("multi-task_ecfp_models/PC3_multi_task_model_ecfp_dn.h5")
    return model_up,model_dn

In [ ]:
model_up, model_dn = read_model()

In [ ]:
def find_target():
    
    gene_target_up = pd.read_csv('harmonizome_dn_binarized_use_for_up_model.csv')
    gene_target_dn = pd.read_csv('harmonizome_up_binarized_use_for_dn_model.csv')
    gene_target_up = gene_target_up[gene_target_up['disease1'] == 'Cancer of prostate_Prostate_GSE1413']
    gene_target_dn = gene_target_dn[gene_target_dn['disease1'] == 'Cancer of prostate_Prostate_GSE1413']
    
    aranan_up = {}
    for i in gene_target_up.columns:
        for j in genes_up:
            if(i == j):
                aranan_up[i] = int(gene_target_up[i])
            
    aranan_dn = {}
    for i in gene_target_dn.columns:
        for j in genes_dn:
            if(i == j):
                aranan_dn[i] = int(gene_target_dn[i])
            
   
    return aranan_up, aranan_dn

In [ ]:
aranan_up, aranan_dn = find_target()

In [ ]:
def target2numpy(aranan_up, aranan_up):
    
    new_aranan_up = []
    for q,w in aranan_up.items():
        new_aranan_up.append(w)
    new_aranan_up=np.array(new_aranan_up)

    new_aranan_dn = []
    for q,w in aranan_dn.items():
        new_aranan_dn.append(w)
    new_aranan_dn=np.array(new_aranan_dn)
    
    return new_aranan_up,new_aranan_dn

In [ ]:
new_aranan_up,new_aranan_dn = target2numpy(aranan_up, aranan_up)

In [ ]:
smiles = new_df['smiles'].tolist()
ecfp = new_df['ecfp'].tolist()
zinc_id = new_df['zinc_id'].tolist()

In [ ]:
ecfpler = []
smileslar = []
def sanal_tarama(p,s):
    
    for k in range(0,len(smiles)):
        bin_ecfp = ecfp[p:s]
        ecfp_new = ecfp[p:s]
        smiles_new = smiles[p:s]
        zinc_id_new = zinc_id[p:s]
        temp = np.zeros((1000,1024))
        
        for i in range(0,len(bin_ecfp)):
            for j in range(0,1024):
                temp[i,j] = bin_ecfp[i][j]

        pred_list = [x for x in model_up.predict(temp)]
        y_up = np.zeros((pred_list[0].shape[0], len(pred_list)))
        for i in range(len(pred_list)):
            y_up[:, i] = pred_list[i].flatten()
        y_up = np.where(y_up > 0.5, 1, 0)

        pred_list = [x for x in model_dn.predict(temp)]
        y_dn = np.zeros((pred_list[0].shape[0], len(pred_list)))
        for i in range(len(pred_list)):
            y_dn[:, i] = pred_list[i].flatten()
        y_dn = np.where(y_dn > 0.5, 1, 0)

        for kk in range(0,len(y_dn)):
            if(((y_dn[kk]==new_aranan_dn).all()) & ((y_up==new_aranan_up).all()) ):
                
                ecfpler.append(ecfp_new[i])
                smileslar.append(smiles_new[i])
                zinc.append(zinc_id_new[i])
                
            if(((y_dn[kk]==new_aranan_dn).all()) ):
                print(smiles_new[kk])

            if(((y_up[kk]==new_aranan_up).all()) ):
                print(smiles_new[kk])
        p=s
        s=s+1000
        return ecfpler,smileslar, zinc

In [ ]:
ecfpler,smileslar, zinc = sanal_tarama(0,1000)